In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url_pre = "http://ballerstein17.league.fantasy.nfl.com/research/pointsagainst?position="
url_suf = "&sort=pointsAgainst_pts&statCategory=pointsAgainst&statSeason=2016&statType=weekPointsAgainst&statWeek="
df = pd.DataFrame()
pos_list = ['QB','RB','WR','TE','K','DEF']
for pos in range(len(pos_list)):
    url_base = url_pre + str(pos) + url_suf
    position = pos_list[pos]
    for week in range(17):
        page = requests.get(url_base+str(week+1))
        soup_list = BeautifulSoup(page.content, 'html.parser').find_all('tr')
        for i in soup_list:
            row = i.find('td', class_='teamNameAndInfo')
            if row != None:
                team = list(row.find('div').children)[1]
                opp = i.find('span', class_='pointsAgainstStatId-opponent').get_text()
                pts = i.find('span', class_='pointsAgainstStatId-pts').get_text()
                df_team = pd.DataFrame({'Team':[team], 'Position':[position], 'Week':[(week+1)], 'Year':2016, 'Opponent':[opp], 'Points_Against':[pts]})
                df = pd.concat([df, df_team])

In [7]:
for pos in df.Position.unique():
    for team in df.Team.unique():
        ave = df[(df.Team == team) & (df.Position == pos)].Points_Against.astype(str).astype(float).sum() / 16
        std = df[(df.Team == team) & (df.Position == pos)].Points_Against.astype(str).astype(float).std()
        df.loc[(df.Team == team) & (df.Position == pos), 'Season_Ave'] = ave
        df.loc[(df.Team == team) & (df.Position == pos), 'Season_STD'] = std

In [9]:
df.to_csv('points_against.csv')

In [11]:
df[(df.Opponent=='@GB') & (df.Team=='Detroit Lions Defense')]

,Opponent,Points_Against,Position,Team,Week,Year,Season_Ave,Season_STD
0,@GB,26.40,QB,Detroit Lions Defense,3,2016,18.97750,9.830612
0,@GB,26.40,RB,Detroit Lions Defense,3,2016,18.97750,9.830612
0,@GB,11.20,WR,Detroit Lions Defense,3,2016,16.09375,5.820021
0,@GB,33.70,TE,Detroit Lions Defense,3,2016,24.43125,11.217869
0,@GB,9.70,K,Detroit Lions Defense,3,2016,9.69375,6.776294
